# Lab 2: Regular Expressions, Edit Distance, and Subword Unit Tokenization

In this lab, we will cover key topics, including regular expressions, edit distance, and subword unit tokenization using Byte Pair Encoding (BPE). This notebook contains exercises that students will need to complete in the code sections. The necessary files will also be provided with this notebook.


## Exercise 1: Basic Regular Expressions

Write regular expressions for the following languages:
1. The set of all alphabetic strings
2. The set of all lower-case alphabetic strings ending in the letter b
3. The set of all strings from the alphabet a,b such that each letter a is immediately preceded by and immediately followed by the letter b

In [ ]:
import re

# 1. The set of all alphabetic strings
regex_1 = r'^[a-zA-Z]+$'

# 2. The set of all lower-case alphabetic strings ending in the letter b
regex_2 = r'^[a-z]*b$'

# 3. The set of all strings from the alphabet a,b such that each letter a is immediately preceded by and immediately followed by the letter b
regex_3 = r'^(b(ab)*)*$'

# Test the regular expressions
test_strings = ["",'Hello', 'world', 'abc', 'bab', 'abab', 'babbabbab', 'bababab', 'abba']

for i, (regex,question) in enumerate([(regex_1,"all alphabetic strings"), (regex_2,"all lower-case alphabetic strings ending in a b"), (regex_3,"all strings from the alphabet a,b such that each a is immediately preceded by and immediately followed by a b")], 1):
    print(f"\nTesting regex {regex} which searches for {question}:")
    for test_str in test_strings:
        print(f"\"{test_str}\" matches: {bool(re.match(regex, test_str))}")


Testing regex ^[a-zA-Z]+$ which searches for all alphabetic strings:
"" matches: False
"Hello" matches: True
"world" matches: True
"abc" matches: True
"bab" matches: True
"abab" matches: True
"babbabbab" matches: True
"bababab" matches: True
"abba" matches: True

Testing regex ^[a-z]*b$ which searches for all lower-case alphabetic strings ending in a b:
"" matches: False
"Hello" matches: False
"world" matches: False
"abc" matches: False
"bab" matches: True
"abab" matches: True
"babbabbab" matches: True
"bababab" matches: True
"abba" matches: False

Testing regex ^(b(ab)*)*$ which searches for all strings from the alphabet a,b such that each a is immediately preceded by and immediately followed by a b:
"" matches: True
"Hello" matches: False
"world" matches: False
"abc" matches: False
"bab" matches: True
"abab" matches: False
"babbabbab" matches: True
"bababab" matches: True
"abba" matches: False


## Exercise 2: Advanced Regular Expressions

Write regular expressions for the following languages. By "word", we mean an alphabetic string separated from other words by whitespace, any relevant punctuation, line breaks, and so forth.

1. The set of all strings with two consecutive repeated words (e.g., "Humbert Humbert" and "the the" but not "the bug" or "the big bug").
2. All strings that start at the beginning of the line with an integer and that end at the end of the line with a word.
3. All strings that have both the word grotto and the word raven in them (but not, e.g., words like grottos that merely contain the word grotto).
4. Write a pattern that captures the first word of an English sentence along with any ending punctuation if it is concatenated to it.

In [ ]:
import re

# 1. Two consecutive repeated words
regex_1 = r'\b(\w+)\s+\1\b' # \b is the word boundry, \1 is the first captured group

# 2. Starts with integer, ends with word
regex_2 = r'^\d+.*\b[a-zA-Z]+\b$'

# 3. Contains both 'grotto' and 'raven' as whole words
regex_3 = r'\b(?:grotto\b.*\braven|raven\b.*\bgrotto)\b'

# 4. First word of an English sentence
regex_4 = r'^([A-Za-z]+)[^\w\s]?'  # punctuation is a character that is not a word character and not a space

# Test the regular expressions
test_strings = [
    "Humbert Humbert is a character character",
    "The big bug",
    "42 is the answer to everything",
    "The raven visited the grotto",
    "Hello, world! How are you?"
]

for i, regex in enumerate([regex_1, regex_2, regex_3, regex_4], 1):
    print(f"\nTesting regex {i}:")
    for test_str in test_strings:
        match = re.search(regex, test_str)
        if match:
            print(f"\"{test_str}\" matches: {match.group()}")
        else:
            print(f"\"{test_str}\" does not match")


Testing regex 1:
"Humbert Humbert is a character character" matches: Humbert Humbert
"The big bug" does not match
"42 is the answer to everything" does not match
"The raven visited the grotto" does not match
"Hello, world! How are you?" does not match

Testing regex 2:
"Humbert Humbert is a character character" does not match
"The big bug" does not match
"42 is the answer to everything" matches: 42 is the answer to everything
"The raven visited the grotto" does not match
"Hello, world! How are you?" does not match

Testing regex 3:
"Humbert Humbert is a character character" does not match
"The big bug" does not match
"42 is the answer to everything" does not match
"The raven visited the grotto" matches: raven visited the grotto
"Hello, world! How are you?" does not match

Testing regex 4:
"Humbert Humbert is a character character" matches: Humbert
"The big bug" matches: The
"42 is the answer to everything" does not match
"The raven visited the grotto" matches: The
"Hello, world! How a

## Exercise 3: Regular Expression for Employee Data Extraction

Using the provided HTML file containing employee information, write regular expressions to extract the following data: Full Name, Email Address, Phone Number, Personal Identification Code, Date of Birth, Website URL, City, Country, Job Title, Department.

Create a regex for each data point and save the extracted employee data in a JSON file.

In [ ]:
import re
import json

# Read the HTML content from the file
html_content = open("employee-data-html.html", "r", encoding='utf-8').read()

department_pattern = r'<div class="department-section" id="(.+?)">.*?<h2>(.*?)</h2>.*?<div class="employee-grid">(.*?)</div>\s*</div>'

employees_list = []

# Find all departments and their respective employee cards
for department_match in re.finditer(department_pattern, html_content, re.DOTALL):
    department_id = department_match.group(1)
    department_name = department_match.group(2)
    employees_section = department_match.group(3)

    # regex patterns for employee cards
    employee_pattern = r'<div class="employee-card">\s*<h3>(.+?)</h3>\s*' \
                       r'<p>Nom:\s*<span class="firstname">(.+?)</span>\s*<span class="lastname">(.+?)</span></p>\s*' \
                       r'<p>Email:\s*(.+?)</p>\s*' \
                       r'<p>Téléphone:\s*(.+?)</p>\s*' \
                       r'<p>Code d\'identification:\s*(.+?)</p>\s*' \
                       r'<p>Date de naissance:\s*(.+?)</p>\s*' \
                       r'<p>Site Web:\s*(.+?)</p>\s*' \
                       r'<p>Ville:\s*(.+?),\s*(.+?)</p>'

    # Extract employee data from the section
    for employee_match in re.finditer(employee_pattern, employees_section, re.DOTALL):
        job_title = employee_match.group(1)
        first_name = employee_match.group(2)
        last_name = employee_match.group(3)
        email = employee_match.group(4)
        phone = employee_match.group(5)
        pid = employee_match.group(6)
        dob = employee_match.group(7)
        website = employee_match.group(8)
        city = employee_match.group(9)
        country = employee_match.group(10)

        # Create a dictionary for the employee
        employee_data = {
            "job_title": job_title,
            "full_name": f"{first_name} {last_name}",
            "email": email,
            "phone": phone,
            "pid": pid,
            "dob": dob,
            "website": website,
            "city": city,
            "country": country,
            "department": department_name
        }

        # Add employee data to the list
        employees_list.append(employee_data)

# Save extracted data to a JSON file
with open("employees_data.json", "w", encoding='utf-8') as json_file:
    json.dump(employees_list, json_file, ensure_ascii=False, indent=4)

print("Employee data extracted and saved to employees_data.json.")


Employee data extracted and saved to employees_data.json.


## Exercise 4: Create a Chatbot Inspired by ELIZA

Design a simple chatbot that mimics the style of ELIZA, an early AI program created in 1966 by Joseph Weizenbaum. ELIZA simulated a conversation with a psychotherapist by using pattern matching and simple responses.

In [ ]:
import re

# Enhanced ELIZA-like chatbot
class ElizaChatbot:
    def __init__(self):
        self.response_rules = [
            (r'Quit', r'Goodbye! It was nice talking to you.'),
            (r'hi|hello|hey', r'Hello! How can I assist you today?'),
            (r'how are you\??', r'I am just a computer program, but thanks for asking! How about you?'),
            (r'(\w+) (tired|bored|sad)', r'Why do you feel \2?'),
            (r'why (.*)', r'Why do you ask that?'),
            (r'(\w+)', r'Stories about \1 are always interesting. Can you tell me more?'),
            (r'(.*) (feel|feels) (.*)', r'What makes you feel \3?'),
            (r'I (.*)', r'Why do you think you \1?'),
            (r'(.*)', r'I didn’t quite understand that. Can you elaborate?'),
        ]

    def eliza_response(self, user_input):
        # Check for specific patterns with case insensitivity
        for pattern, response in self.response_rules:
            match = re.match(pattern, user_input, re.IGNORECASE)
            if match:
                return re.sub(pattern, response, user_input, flags=re.IGNORECASE)

        # Return a default response if no patterns matched
        return "I didn't quite understand that. Can you elaborate?"

# Example usage
if __name__ == "__main__":
    eliza = ElizaChatbot()
    print("ELIZA: Hello! I'm your virtual assistant. Type 'Quit' to exit.")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("ELIZA: Goodbye! It was nice talking to you.")
            break
        print("ELIZA:", eliza.eliza_response(user_input))


ELIZA: Hello! I'm your virtual assistant. Type 'Quit' to exit.
ELIZA: Hello! How can I assist you today?
ELIZA: Stories about can are always interesting. Can you tell me more? Stories about you are always interesting. Can you tell me more? Stories about help are always interesting. Can you tell me more? Stories about me are always interesting. Can you tell me more??
ELIZA: Hello! How can I assist you today?
ELIZA: Stories about i are always interesting. Can you tell me more? Stories about am are always interesting. Can you tell me more? Stories about tired are always interesting. Can you tell me more?
ELIZA: Stories about tired are always interesting. Can you tell me more? Stories about and are always interesting. Can you tell me more? Stories about tired are always interesting. Can you tell me more? Stories about and are always interesting. Can you tell me more? Stories about tired are always interesting. Can you tell me more?
ELIZA: Stories about she are always interesting. Can you t

In [ ]:
import re

# Empathetic ELIZA-like chatbot for listening
class ListeningElizaChatbot:
    def __init__(self):
        self.response_rules = [
            (r'quit', r'Goodbye! I’m here for you anytime you need to talk.'),
            (r'hi|hello|hey', r'Hello! I’m here to listen. What’s on your mind?'),
            (r'how are you\??', r'I’m just a program, but I’m here to listen to you. How are you feeling?'),
            (r'(\w+) (tired|bored|sad|angry|frustrated)', r'I can sense that you’re feeling \2. That sounds tough.'),
            (r'I (don\'?t|do not) (want to) (eat|talk)', r"It’s okay to feel that way. Sometimes, we all have moments when we don't feel like doing things. What do you think is causing that?"),
            (r'why (.*)', r'It’s understandable to wonder about that. I’m here to listen.'),
            (r'I (.*)', r'It sounds like you’re feeling something about that: \1. Would you like to share more?'),
            (r'(.*) (feel|feels) (.*)', r'It’s okay to feel \3. I’m here for you.'),
            (r'(\w+)', r'Thank you for sharing that. What else is on your mind?'),
            (r'(.*)', r'I’m here to listen. Please tell me more about what you’re feeling.'),
        ]

    def eliza_response(self, user_input):
        # Check for specific patterns with case insensitivity
        for pattern, response in self.response_rules:
            match = re.match(pattern, user_input, re.IGNORECASE)
            if match:
                return re.sub(pattern, response, user_input, flags=re.IGNORECASE)

        # Return a default response if no patterns matched
        return "I didn’t quite understand that, but I'm here to listen. Can you tell me more?"

# Example usage
if __name__ == "__main__":
    eliza = ListeningElizaChatbot()
    print("ELIZA: Hello! I'm your empathetic listener. Type 'Quit' to exit.")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'quit':
            print("ELIZA: Goodbye! I’m here for you anytime you need to talk.")
            break
        print("ELIZA:", eliza.eliza_response(user_input))


ELIZA: Hello! I'm your empathetic listener. Type 'Quit' to exit.
ELIZA: Hello! I’m here to listen. What’s on your mind?
ELIZA: It’s okay to feel that way. Sometimes, we all have moments when we don't feel like doing things. What do you think is causing that?
ELIZA: Goodbye! I’m here for you anytime you need to talk.


## Exercise 5: Edit Distance

1. Compute the edit distance (using insertion cost 1, deletion cost 1, substitution cost 1) of "leda" to "deal". Show your work (using the edit distance grid).

2. Implement a minimum edit distance algorithm and use your hand-computed results to check your code.

In [1]:
# Function to calculate the minimum edit distance between two strings
def edit_distance(s1, s2):
    m, n = len(s1), len(s2)

    # Create a 2D array to store distances
    dp = [[0] * (n + 1) for _ in range(m + 1)]

    # Initialize the first row and column
    for i in range(m + 1):
        dp[i][0] = i  # Cost of deleting characters from s1
    for j in range(n + 1):
        dp[0][j] = j  # Cost of inserting characters into s1

    # Fill the dp array
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if s1[i - 1] == s2[j - 1]:
                # Characters match, no extra cost
                dp[i][j] = dp[i - 1][j - 1]
            else:
                # Minimum of insert, delete, or substitute
                dp[i][j] = min(
                    dp[i - 1][j] + 1,  # Deletion
                    dp[i][j - 1] + 1,  # Insertion
                    dp[i - 1][j - 1] + 2  # Substitution
                )

    # Return the minimum edit distance
    return dp[m][n], dp

# Function to print the edit distance grid
def print_edit_distance_grid(s1, s2, dp):
    print(f"Edit distance grid for '{s1}' to '{s2}':")
    print("    ", "   ".join(f"{ch}" for ch in " " + s2))
    print("   " + "----" * (len(s2) + 1))
    for i in range(len(dp)):
        row_label = s1[i - 1] if i > 0 else " "
        row = " | ".join(f"{dp[i][j]:>2}" for j in range(len(dp[i])))
        print(f"{row_label} | {row}")
    print()

# Compute and display the edit distance for "leda" to "deal"
source, target = "leda", "deal"
distance, dp_grid = edit_distance(source, target)

# Print the edit distance result and grid
print(f"Minimum edit distance from '{source}' to '{target}': {distance}")

# Print the grid for clarity
print_edit_distance_grid(source, target, dp_grid)


Minimum edit distance from 'leda' to 'deal': 4
Edit distance grid for 'leda' to 'deal':
         d   e   a   l
   --------------------
  |  0 |  1 |  2 |  3 |  4
l |  1 |  2 |  3 |  4 |  3
e |  2 |  3 |  2 |  3 |  4
d |  3 |  2 |  3 |  4 |  5
a |  4 |  3 |  4 |  3 |  4



# Exercise 6: Spell Correction using NLTK

In this exercise, we'll implement spell correction functions using NLTK and the concepts of edit distance.

## Question 1:
Write a function that takes a misspelled word and returns the closest valid word from a given textual corpus using edit distance.

**Hint:** You can use the top 5000 most frequent words from an NLTK Gutenberg text such as 'austen-emma.txt'.

## Question 2:
Use the function you created in Question 1 to correct spelling errors in full sentences.

**Hint:** You can randomly introduce spelling errors into existing sentences from the Gutenberg Corpus available in NLTK by altering long words (those with more than 5 characters) through the insertion of a random character.


## Question 3:
Implement the following steps to evaluate the effectiveness of your spell correction:

1. **Select Sentences**: Choose a set of sentences from the Gutenberg Corpus available in NLTK.
2. **Introduce Spelling Errors**: Randomly alter some long words in each sentence to create spelling errors (through the insertion of a random character). Ensure that only words longer than 5 characters as before.
3. **Correct the Sentences**: Use the function from Question 1 to correct the misspelled words in each altered sentence.
4. **Print Results**: For each original sentence, print the altered sentence and the suggested corrections for the misspelled words.
5. **Calculate Precision**: Evaluate the accuracy of the corrections by calculating the precision of the spell correction method on the provided test set in the code section below.


In [12]:
import nltk
from nltk.corpus import gutenberg, words
from nltk import FreqDist
from nltk.tokenize import word_tokenize
import random
import string

# Step 1: Download necessary NLTK data
nltk.download('punkt')
nltk.download('gutenberg')
nltk.download('words')

# Define the vocab size
Vocab_size = 5000

# Step 2: Load text and tokenize, filtering punctuation and converting to lowercase
tokens = [word.lower() for word in word_tokenize(gutenberg.raw('austen-emma.txt')) if word.isalpha()]
# Get the most frequent words as the vocabulary
fdist = FreqDist(tokens)
vocab = set(word for word, _ in fdist.most_common(Vocab_size))

# Function to find the closest word using edit distance
def correct_spelling(word, vocab):
    closest_word = min(vocab, key=lambda v: edit_distance(word, v))
    return closest_word

# Function to introduce spelling errors
def introduce_spelling_errors(sentence, error_rate=0.3, word_length=5):
    words = sentence.split()
    new_sentence = []
    for word in words:
        if len(word) > word_length and random.random() < error_rate:
            pos = random.randint(0, len(word) - 1)
            char = random.choice(string.ascii_lowercase)
            word = word[:pos] + char + word[pos:]
        new_sentence.append(word)
    return " ".join(new_sentence)

# Function to correct a sentence
def correct_sentence(sentence, vocab):
    corrected_sentence = []
    for word in sentence.split():
        if word.lower() in vocab:
            corrected_sentence.append(word)
        else:
            corrected_word = correct_spelling(word.lower(), vocab)
            corrected_sentence.append(corrected_word)
    return " ".join(corrected_sentence)

# Define test sentences
test_sentences = [
    "She was pleased to have such a friend.",
    "He thought it was a great opportunity.",
    "The sun was shining brightly in the sky.",
    "They were walking towards the market together.",
    "I hope you will come to the party."
]

# Introduce errors, correct sentences, and store results
altered_sentences = [introduce_spelling_errors(sentence) for sentence in test_sentences]
corrected_sentences = [correct_sentence(sentence, vocab) for sentence in altered_sentences]

# Display original, altered, and corrected sentences
for original, altered, corrected in zip(test_sentences, altered_sentences, corrected_sentences):
    print("Original sentence:   ", original)
    print("Altered sentence:    ", altered)
    print("Corrected sentence:  ", corrected)
    print()

# Calculate precision by comparing original to corrected sentences
def calculate_precision(original_sentences, corrected_sentences):
    total_words = 0
    correct_matches = 0

    for original_sentence, corrected_sentence in zip(original_sentences, corrected_sentences):
        original_words = original_sentence.split()
        corrected_words = corrected_sentence.split()

        # Define a translation table to remove punctuation
        translator = str.maketrans('', '', string.punctuation)

        for orig_word, corr_word in zip(original_words, corrected_words):
            orig_word_clean = orig_word.translate(translator).lower()
            corr_word_clean = corr_word.translate(translator).lower()

            total_words += 1
            if corr_word_clean == orig_word_clean:
                correct_matches += 1

    # Calculate precision manually
    precision = correct_matches / total_words if total_words > 0 else 0
    return precision

# Calculate and print precision of the spell correction function
precision = calculate_precision(test_sentences, corrected_sentences)
print(f"Precision of spell correction: {precision:.2f}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


Original sentence:    She was pleased to have such a friend.
Altered sentence:     She was pleased to have such a friend.
Corrected sentence:   She was pleased to have such a friend

Original sentence:    He thought it was a great opportunity.
Altered sentence:     He thought it was a great oppgortunity.
Corrected sentence:   He thought it was a great opportunity

Original sentence:    The sun was shining brightly in the sky.
Altered sentence:     The sun was shining brightly in the sky.
Corrected sentence:   The sun was singing rightly in the sky

Original sentence:    They were walking towards the market together.
Altered sentence:     They were waslking towards the market togetheer.
Corrected sentence:   They were walking towards the market together

Original sentence:    I hope you will come to the party.
Altered sentence:     I hope you will come to the party.
Corrected sentence:   I hope you will come to the party

Precision of spell correction: 0.95


In [15]:
# Introduce errors, correct sentences, and store results
altered_sentences = [introduce_spelling_errors(sentence, error_rate=0.7, word_length=2) for sentence in test_sentences]
corrected_sentences = [correct_sentence(sentence, vocab) for sentence in altered_sentences]

# Display original, altered, and corrected sentences
for original, altered, corrected in zip(test_sentences, altered_sentences, corrected_sentences):
    print("Original sentence:   ", original)
    print("Altered sentence:    ", altered)
    print("Corrected sentence:  ", corrected)
    print()

# Calculate and print precision of the spell correction function
precision = calculate_precision(test_sentences, corrected_sentences)
print(f"Precision of spell correction: {precision:.2f}")


Original sentence:    She was pleased to have such a friend.
Altered sentence:     Shre was pleased to hahve sucyh a friendu.
Corrected sentence:   she was pleased to have such a friend

Original sentence:    He thought it was a great opportunity.
Altered sentence:     He thougtht it ewas a greadt oppordtunity.
Corrected sentence:   He thought it was a great opportunity

Original sentence:    The sun was shining brightly in the sky.
Altered sentence:     The stun wars shininng brightly in ethe skhy.
Corrected sentence:   The sun war singing rightly in the sky

Original sentence:    They were walking towards the market together.
Altered sentence:     They weore ewalking toowards the mamrket together.
Corrected sentence:   They were walking towards the market together

Original sentence:    I hope you will come to the party.
Altered sentence:     I hope you wilvl come to the party.
Corrected sentence:   I hope you will come to the party

Precision of spell correction: 0.92


# Exercise 7: Implementing Byte Pair Encoding (BPE) Tokenization

In this exercise, you will implement the Byte Pair Encoding (BPE) tokenization method step by step. Follow the instructions below to build your BPE tokenizer.

### Step 1: Get Vocabulary

Implement the function `get_vocab(sentences)` that takes a list of sentences and returns a dictionary with the counts of each token in the vocabulary.

### Step 2: Get Stats

Implement the function `get_stats(vocab)` that takes the vocabulary dictionary and returns a dictionary of pairs of tokens and their frequencies.

### Step 3: Merge Vocabulary

Implement the function `merge_vocab(vocab, pair)` that takes the vocabulary and a pair of tokens to merge. It should return the updated vocabulary.

### Step 4: BPE

Implement the function `bpe(sentences, num_merges)` that uses the previously defined functions to perform BPE tokenization on the provided sentences.

### Step 5: Apply BPE

Implement the function `apply_bpe(sentence, vocab)` that takes a sentence and the vocabulary and returns the tokenized version of the sentence using BPE.

Once you have completed all steps, test your implementation with sample sentences to observe how the BPE tokenization works. You can adjust the number of merges to see how it affects the tokenization.


In [ ]:
from collections import defaultdict
import nltk
nltk.download('punkt')  # Ensure the Punkt tokenizer is downloaded

# Step 1: Get Vocabulary
def get_vocab(sentences):
    """
    Functionality:
    - This function processes a list of sentences and generates a vocabulary where each word
        is split into individual characters (with an end-of-word marker '</w>'). The vocabulary
        is represented as a dictionary where keys are the tokenized words and values are their
        frequencies.
    - '</w>' is added at the end of each word. This is an "end-of-word" marker, which helps the
         algorithm distinguish between the end of a word and the beginning of the next one.
         It's crucial for maintaining word boundaries during the merging process.
    -  Example:
    sentences = ["low", "lowest"]
    Output vocab: { "l o w </w>": 1, "l o w e s t </w>": 1 }

    Input:
    - sentences: A list of strings (sentences or words) to be processed into the vocabulary.

    Output:
    - vocab: A dictionary where the keys are tokenized versions of words (character-level BPE tokens),
             and the values are the frequency of each word in the input sentences.
    """
    vocab = defaultdict(int)

   # your code here (don't forget to toknize each sent)
    vocab = defaultdict(int)

    for sentence in sentences:
        for word in sentence.lower().split():
            tokenized_word = ' '.join(list(word)) + ' </w>'
            vocab[tokenized_word] += 1

    return vocab

# Step 2: Get Stats
def get_stats(vocab):
    """
    Functionality:
    - This function computes the frequency of each adjacent pair of tokens (bigrams) from the vocabulary.
    - It identifies which pairs of tokens (characters) appear together most frequently.
    - The '</w>' token is treated like any other character in this process.
    Example:
    input vocab = { "l o w </w>": 1, "l o w e s t </w>": 1 }
    output pairs ==> pairs = { ('l', 'o'): 2, ('o', 'w'): 2, ('w', '</w>'): 1, ('w', 'e'): 1, ('e', 's'): 1, ('s', 't'): 1, ('t', '</w>'): 1 }
    Input:
    - vocab: A dictionary where keys are tokenized words (character sequences) and values are frequencies of the words.

    Output:
    - pairs: A dictionary where the keys are pairs of consecutive tokens, and the values are the sum of their
             frequencies across all the words in the vocabulary.
    """
    pairs = defaultdict(int)

    # your code here
    pairs = defaultdict(int)

    for word, freq in vocab.items():
        tokens = word.split()
        for i in range(len(tokens) - 1):
            pair = (tokens[i], tokens[i + 1])
            pairs[pair] += freq

    return pairs

# Step 3: Merge Vocabulary
def merge_vocab(vocab, pair):
    """
    Functionality:
    This function merges the most frequent pair of tokens in the vocabulary by replacing the pair
    with a single combined token. It updates the vocabulary to reflect this merge.
    Example:
        old vocab = { "l o w </w>": 1, "l o w e s t </w>": 1}
        freq pair = ('o', 'w')
        ==> new vocab = { "l ow </w>": 1, "l ow e s t </w>": 1 }

    Input:
    - vocab: A dictionary where keys are tokenized words (character sequences) and values are frequencies of the words.
    - pair: A tuple containing two tokens (characters) that should be merged.

    Output:
    - new_vocab: A new dictionary where the specified token pair is merged in each word in the vocabulary.
    """
    new_vocab = {}

    # your code here
    pattern = re.compile(r'(?<!\S)' + re.escape(' '.join(pair)) + r'(?!\S)')

    for word, freq in vocab.items():
        new_word = pattern.sub(''.join(pair), word)
        new_vocab[new_word] = freq

    return new_vocab

# Step 4: BPE (Byte Pair Encoding)
def bpe(sentences, num_merges):
    """
    Functionality:
    This function performs the BPE algorithm on a set of sentences, iteratively merging the most
    frequent pairs of tokens until the specified number of merges is reached.

    Input:
    - sentences: A list of strings (sentences or words) to be processed into BPE tokens.
    - num_merges: The number of merges to perform (i.e., how many token pairs to combine).

    Output:
    - vocab: The updated vocabulary after performing the BPE merges.
    - merges: A list of the token pairs that were merged during the BPE process.
    """
    vocab = get_vocab(sentences)  # Initialize vocabulary by tokenizing words into characters
    merges = []  # To keep track of the merges performed

    # your code here
    for _ in range(num_merges):
        pairs = get_stats(vocab)
        if not pairs:
            break

        best_pair = max(pairs, key=pairs.get)
        vocab = merge_vocab(vocab, best_pair)
        merges.append(best_pair)

    return vocab, merges

# Step 5: Apply BPE
def apply_bpe(sentence, merges):
    """
    Functionality:
    This function tokenizes a new sentence using the learned BPE merges. It progressively applies
    each merge to the sentence in the order they were learned.

    Input:
    - sentence: A string (word or sentence) to be tokenized using the BPE merges.
    - merges: A list of tuples representing the token pairs that were merged during BPE training.

    Output:
    - A list of tokens representing the sentence after applying BPE tokenization.
    """

    # your code here
    tokens = list(sentence.lower()) + ['</w>']

    for merge in merges:
        i = 0
        while i < len(tokens) - 1:
            if (tokens[i], tokens[i + 1]) == merge:
                tokens[i:i + 2] = [''.join(merge)]
            else:
                i += 1

    return tokens  # Return the final tokenized sentence as a list of tokens

# Example Usage
if __name__ == "__main__":
    # Sample sentences for BPE training
    sentences = [
        "Low",
        "lowest",
        "the",
        "this",
        "fastest"
    ]
    # Perform BPE with a specified number of merges (e.g., 10)
    num_merges = 10
    vocab, merges = bpe(sentences, num_merges)

    print("Vocabulary after BPE:", vocab)  # Vocabulary after applying the BPE merges
    print("Learned merges:", merges)  # List of merges learned during BPE training

    # Apply BPE to a new sentence that wasn't part of the original sentences
    sentence = "lowder in the"
    tokenized_sentence = apply_bpe(sentence, merges)

    print("Tokenized sentence:", tokenized_sentence)  # Final tokenized version of the input sentence


Vocabulary after BPE: defaultdict(<class 'int'>, {})
Learned merges: []
Tokenized sentence: []


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Exercise 8: Using SentencePiece for Byte Pair Encoding (BPE) Tokenization

In this exercise, you will learn how to use the SentencePiece library to implement Byte Pair Encoding (BPE) for tokenization. Follow the steps below to tokenize text using BPE.

### Step 1: Install SentencePiece

Ensure that you have the SentencePiece library installed in your Python environment. You can install it using pip:

```python
!pip install sentencepiece
```

### Step 2: Prepare Your Text
Load and merge all texts from the Gutenberg corpus available in NLTK. You can use the following code snippet to achieve this:

### Step 3: Train the BPE Model
Utilize SentencePiece to train a BPE model on the merged text. Specify the vocabulary size you want to achieve during the training process. For example:
```python
import sentencepiece as spm

# Train the SentencePiece model
spm.SentencePieceTrainer.Train('--input=merged_gutenberg.txt --model_prefix=m --vocab_size=5000')
```


### Step 4: Tokenize the Text
Once the model is trained, use it to tokenize your text into subword units. You can test the tokenizer with the following sample sentences:

```python
# Load the trained model
sp = spm.SentencePieceProcessor(model_file='m.model')

# Tokenize a sentence
tokens = sp.encode(sentence, out_type=str)
```

### Step 5: What about Arabic Texts !
Try to do the same thing but this time using an Arabic text corpus.

In [ ]:
# Step 1: Install SentencePiece
!pip install sentencepiece

# Step 2: Prepare Your Text
import nltk
from nltk.corpus import gutenberg

# Download the Gutenberg corpus if not already downloaded
nltk.download('gutenberg')

# Get the list of file IDs from the Gutenberg corpus
file_ids = gutenberg.fileids()

# Merge all texts into a single string
merged_text = ""
for file_id in file_ids:
    merged_text += gutenberg.raw(file_id) + "\n"

# Write the merged text to a temporary file
with open('merged_gutenberg.txt', 'w', encoding='utf-8') as f:
    f.write(merged_text)


[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


## Main differences between our simple BPE implementation and SentencePiece BPE:

### 1. End-of-word marker:
- **Simple BPE:** Uses `</w>` at the end of each word.
- **SentencePiece:** Uses `_` (underscore) at the beginning of each word.

**Example:**
- Simple BPE: `"low</w> est</w>"`
- SentencePiece: `"_low _est"`

---

### 2. Tokenization approach:
- **Simple BPE:** Tokenizes words first, then applies BPE.
- **SentencePiece:** Treats the entire input as a sequence of characters, including spaces.

**Example:**
- Simple BPE: `["low</w>", "est</w>"]`
- SentencePiece: `["", "l", "o", "w", "", "e", "s", "t"]`

---

### 3. Handling of whitespace:
- **Simple BPE:** Whitespace is removed during tokenization.
- **SentencePiece:** Preserves whitespace as part of the tokenization process.

**Example:**
- Simple BPE: `"hello world"` -> `["hello</w>", "world</w>"]`
- SentencePiece: `"hello world"` -> `["_hello", "_world"]`

---

### 4. Subword regularization:
- **Simple BPE:** No built-in regularization.
- **SentencePiece:** Supports subword regularization techniques like BPE-dropout.

---

### 5. Unicode handling:
- **Simple BPE:** Limited Unicode support (depends on NLTK tokenization).
- **SentencePiece:** Full Unicode support, treating each Unicode character as a basic unit.

---

### 6. Vocabulary size control:
- **Simple BPE:** Controlled by number of merge operations.
- **SentencePiece:** Directly specify desired vocabulary size.

---

### 7. Out-of-vocabulary (OOV) handling:
- **Simple BPE:** No specific OOV handling.
- **SentencePiece:** Has built-in OOV handling, often using a special `<unk>` token.

---

### 8. Reversibility:
- **Simple BPE:** Not easily reversible due to `</w>` placement.
- **SentencePiece:** Easily reversible due to `_` placement at word beginnings.

**Example:**
- SentencePiece: `"_app le"` -> `"apple"` (easily reversible)
- Simple BPE: `"app le</w>"` -> `"apple"` (requires special handling of `</w>`)

---

### 9. Efficiency and speed:
- **Simple BPE:** Generally very slow, especially for large datasets.
- **SentencePiece:** Highly optimized for speed and efficiency.
